In [2]:
%load_ext autoreload
%autoreload 2
import gc
from glob import glob
import os
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import timer, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename, logger_func
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature
import shutil

In [26]:
feim = pd.read_csv('../output/valid_single_feature_tochi.csv', header=None)
# feim = pd.read_csv('../output/valid_single_feature.csv', header=None)
feim.columns = ['datetime', 'DT-M', 'feature', 'score']

feim = feim[feim['datetime']>='20190915_00000']
print(feim['datetime'].max())

feim.set_index('feature', inplace=True)
cnt = feim.groupby('feature')['score'].count()
idx = cnt[cnt>=3].index
feim = feim.loc[idx]
may = feim[feim['DT-M']=='2018-5']
may.sort_values(by='score', ascending=False, inplace=True)
feim.loc[may.index]

20190915_11572


,datetime,DT-M,score
feature,,,
524__combi3__card3-addr1-R_emaildomain_suffix_fraud_mean_train,20190915_10581,2018-5,0.936685
524__combi3__card3-addr1-R_emaildomain_suffix_fraud_mean_train,20190915_10581,2018-4,0.949322
524__combi3__card3-addr1-R_emaildomain_suffix_fraud_mean_train,20190915_10581,2018-3,0.932578
524__combi4__card1-addr1-addr2-cents_fraud_mean_train,20190915_11321,2018-5,0.936350
524__combi4__card1-addr1-addr2-cents_fraud_mean_train,20190915_11321,2018-4,0.947986
524__combi4__card1-addr1-addr2-cents_fraud_mean_train,20190915_11321,2018-3,0.935188
524__combi5__ProductCD-card1-card3-addr1-cents_fraud_mean_train,20190915_11202,2018-5,0.936081
524__combi5__ProductCD-card1-card3-addr1-cents_fraud_mean_train,20190915_11202,2018-4,0.947288
524__combi5__ProductCD-card1-card3-addr1-cents_fraud_mean_train,20190915_11202,2018-3,0.935230


In [21]:
tmp = feim[feim['DT-M']=='2018-5']
list_feature = tmp[tmp['score']>0.937].index
from_dir = 'product_feature'
# from_dir = 'check_trush'
# from_dir = 'org_trush'
# to_dir = 'valid_use'

for feature in list_feature:
    print(feature)
    move_feature([feature.replace('_train', '')], from_dir, to_dir)

509__C11-V44__ratio__ProductCD-C_train
512__V127-V318__ratio__ProductCD-W_train
512__V127-V78__diff__ProductCD-C_train
512__V130-V281__diff__ProductCD-H_train
512__V53-V78__diff__ProductCD-W_train
603__addr1-addr2_C8__ProductCD-H_std_train
603__card1-card4_C6__ProductCD-C_mean_train
603__card3-card6_C9__ProductCD-C_std_train
603__card6-addr2_C4__ProductCD-C_std_train


In [20]:
tmp[tmp['score']>0.937]

,datetime,DT-M,score
feature,,,
509__C11-V44__ratio__ProductCD-C_train,20190914_06201,2018-5,0.937060
512__V127-V318__ratio__ProductCD-W_train,20190914_13211,2018-5,0.938651
512__V127-V78__diff__ProductCD-C_train,20190914_16115,2018-5,0.937358
512__V130-V281__diff__ProductCD-H_train,20190914_06140,2018-5,0.937180
512__V53-V78__diff__ProductCD-W_train,20190914_13155,2018-5,0.937122
603__addr1-addr2_C8__ProductCD-H_std_train,20190914_16063,2018-5,0.937199
603__card1-card4_C6__ProductCD-C_mean_train,20190914_07480,2018-5,0.937604
603__card3-card6_C9__ProductCD-C_std_train,20190914_06595,2018-5,0.937283
603__card6-addr2_C4__ProductCD-C_std_train,20190914_12505,2018-5,0.937022
